In [ ]:
# --- STEP 1: Install dependencies ---
!pip install yt-dlp pydub tqdm ipywidgets > /dev/null

# --- STEP 2: Import libraries ---
import os
import subprocess
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import ipywidgets as widgets
from IPython.display import display, FileLink

# --- STEP 3: File upload widget ---
upload_widget = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='Upload song list (.txt)'
)
display(upload_widget)

def process_upload(change):
    if not upload_widget.value:
        print("Please upload a file.")
        return

    uploaded_file = list(upload_widget.value.values())[0]
    filename = uploaded_file['metadata']['name']

    with open(filename, 'wb') as f:
        f.write(uploaded_file['content'])

    # Continue workflow
    run_downloader(filename)

upload_widget.observe(process_upload, names='value')

# --- STEP 4: Define downloader function ---
def run_downloader(filename):
    # --- Read search terms ---
    with open(filename, 'r', encoding='utf-8') as f:
        search_terms = [line.strip() for line in f if line.strip()]

    print(f"🎶 Found {len(search_terms)} songs in {filename}")

    # --- Prepare output folder ---
    output_dir = "mp3_downloads"
    os.makedirs(output_dir, exist_ok=True)

    # --- Download function ---
    def download_song(term):
        safe_name = term.replace(" ", "_")[:60]
        existing_files = [f for f in os.listdir(output_dir) if safe_name.lower() in f.lower()]

        if existing_files:
            return (term, "skipped")

        cmd = [
            "yt-dlp",
            f"ytsearch1:{term}",
            "-x", "--audio-format", "mp3",
            "--audio-quality", "0",
            "-o", f"{output_dir}/%(title)s.%(ext)s",
            "--restrict-filenames"
        ]

        try:
            subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            return (term, "success")
        except subprocess.CalledProcessError:
            return (term, "failed")

    # --- Parallel downloads ---
    success, failed, skipped = [], [], []
    print("\n⚡ Starting multi-threaded downloads...\n")

    MAX_WORKERS = 4
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(download_song, term): term for term in search_terms}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Progress", unit="song"):
            term, status = future.result()
            if status == "success":
                success.append(term)
            elif status == "failed":
                failed.append(term)
            else:
                skipped.append(term)

    # --- Summary ---
    print("\n✅ Downloading complete!\n")
    print(f"📊 Summary:")
    print(f"  ✔️ Successful: {len(success)}")
    print(f"  ⏭️ Skipped: {len(skipped)}")
    print(f"  ❌ Failed: {len(failed)}")

    if failed:
        print("\n⚠️ Could not download:")
        for f in failed:
            print(f"   - {f}")

    # --- Zip results ---
    subprocess.run(["zip", "-r", "-q", "mp3_downloads.zip", output_dir])
    print("\n📦 All done! Download your zip file below:")

    display(FileLink("mp3_downloads.zip"))